In [ ]:
!unzip "/content/ToothNumber_TaskDataset.zip" -d /content/ToothNumber_TaskDataset

Archive:  /content/ToothNumber_TaskDataset.zip
   creating: /content/ToothNumber_TaskDataset/images/
   creating: /content/ToothNumber_TaskDataset/labels/
  inflating: /content/ToothNumber_TaskDataset/images/cate8-00059_jpg.rf.d0bd3cb39cc429692abe9c8cf6f9c75e.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/b879ef2c-20250503-101257822.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/cate1-00008_jpg.rf.10436fa8d35c0764df5402f3a6f093dc.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/1c50bda9-20241231-145320490.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/cate8-00131_jpg.rf.9755ad7a2010d677500a57fe7291285f.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/b9b8b22f-20240730-123647335.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/cate2-00127_jpg.rf.aaba4e5e69cf5a828885a05115aa8d11.jpg  
  inflating: /content/ToothNumber_TaskDataset/images/cate8-00191_jpg.rf.856a6afd259fc24434201fdf1c9b37a0.jpg  
  inflating: /content/ToothNu

In [ ]:
!zip -r YOLO12m.zip /content/runs/detect/YOLO12m_predected

updating: content/runs/detect/YOLO12m_predected/ (stored 0%)
updating: content/runs/detect/YOLO12m_predected/cate7-00105_jpg.rf.daf2fd9f3c80a1d37415747e5e83a068.jpg (deflated 3%)
updating: content/runs/detect/YOLO12m_predected/3a483ad5-20240827-120738078.jpg (deflated 2%)
updating: content/runs/detect/YOLO12m_predected/cate2-00003_jpg.rf.ee753866ade30285aa03556cdfbb28d5.jpg (deflated 3%)
updating: content/runs/detect/YOLO12m_predected/cate1-00029_jpg.rf.97fe17dca11f631d02b8290db8b4c8de.jpg (deflated 3%)
updating: content/runs/detect/YOLO12m_predected/cate5-00019_jpg.rf.0bdea403aa040379bfc678546ea3137e.jpg (deflated 4%)
updating: content/runs/detect/YOLO12m_predected/cate8-00267_jpg.rf.8303c47ab0c59b7d7ae202f0041fcf20.jpg (deflated 2%)
updating: content/runs/detect/YOLO12m_predected/cate2-00024_jpg.rf.fcdad1d1e96e782bead30ea33af7ed9f.jpg (deflated 3%)
updating: content/runs/detect/YOLO12m_predected/cate5-00084_jpg.rf.bf231961b5cf86ba29e10c1a195514b6.jpg (deflated 3%)
updating: content/r

In [ ]:
from google.colab import files
files.download("YOLO12m_predected.zip")


FileNotFoundError: Cannot find file: YOLO12m_predected.zip

In [1]:
!pip install ultralytics --upgrade
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
!nvidia-smi

CUDA available: True
Device count: 1
GPU name: Tesla T4
Sat Aug 30 08:17:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                

In [4]:
import os
import random
import shutil
import yaml

# ===== CONFIG =====
SOURCE_IMAGES = "ToothNumber_TaskDataset/images"
SOURCE_LABELS = "ToothNumber_TaskDataset/labels"
OUTPUT_DIR = "dental_dataset"
YAML_FILE = "data.yaml"
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
# ==================

# Create output directories
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(f"{OUTPUT_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/labels/{split}", exist_ok=True)

# Get all image files
images = [f for f in os.listdir(SOURCE_IMAGES) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(images)

# Split data
train_end = int(TRAIN_SPLIT * len(images))
val_end = train_end + int(VAL_SPLIT * len(images))

train_files = images[:train_end]
val_files = images[train_end:val_end]
test_files = images[val_end:]

dataset_splits = {
    "train": train_files,
    "val": val_files,
    "test": test_files
}

# Copy images and labels
for split, files in dataset_splits.items():
    for img_file in files:
        label_file = os.path.splitext(img_file)[0] + ".txt"
        shutil.copy(os.path.join(SOURCE_IMAGES, img_file), f"{OUTPUT_DIR}/images/{split}/{img_file}")
        shutil.copy(os.path.join(SOURCE_LABELS, label_file), f"{OUTPUT_DIR}/labels/{split}/{label_file}")

print(f"✅ Dataset split completed: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

# Create data.yaml
class_names = [
    "Canine (13)", "Canine (23)", "Canine (33)", "Canine (43)",
    "Central Incisor (21)", "Central Incisor (41)", "Central Incisor (31)", "Central Incisor (11)",
    "First Molar (16)", "First Molar (26)", "First Molar (36)", "First Molar (46)",
    "First Premolar (14)", "First Premolar (34)", "First Premolar (44)", "First Premolar (24)",
    "Lateral Incisor (22)", "Lateral Incisor (32)", "Lateral Incisor (42)", "Lateral Incisor (12)",
    "Second Molar (17)", "Second Molar (27)", "Second Molar (37)", "Second Molar (47)",
    "Second Premolar (15)", "Second Premolar (25)", "Second Premolar (35)", "Second Premolar (45)",
    "Third Molar (18)", "Third Molar (28)", "Third Molar (38)", "Third Molar (48)"
]

yaml_dict = {
    "train": f"{OUTPUT_DIR}/images/train",
    "val": f"{OUTPUT_DIR}/images/val",
    "test": f"{OUTPUT_DIR}/images/test",
    "names": {i: name for i, name in enumerate(class_names)}
}

with open(YAML_FILE, "w") as f:
    yaml.dump(yaml_dict, f)

print(f"✅ data.yaml created at: {YAML_FILE}")

✅ Dataset split completed: 397 train, 49 val, 51 test
✅ data.yaml created at: data.yaml


In [10]:
from ultralytics import YOLO

model = YOLO("yolov10s.pt")   # Load pretrained small model
results = model.train(
    data="data.yaml",  # Update path to data.yaml
    imgsz=640,
    epochs=25,    # Start with small number for testing
    batch=16,    # Increase batch size for GPU (T4 can handle more than CPU)
    device=0
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False

In [11]:
# Validate the model
metrics = model.val()
print(metrics.box.map)  # Print mAP@0.5:0.95
print(metrics.box.map50)  # Print mAP@0.5

# Perform inference on a test image
results = model.predict(source="dental_dataset/images/test",
    save=True
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10s summary (fused): 106 layers, 7,230,384 parameters, 0 gradients, 21.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1399.9±423.5 MB/s, size: 78.3 KB)
val: Scanning /content/dental_dataset/labels/val.cache... 201 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 201/201 339667.6it/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 33, len(boxes) = 5637. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 1.1it/s 11.6s
                   all        201       5637      0.846      0.847      0.916      0.661
           Canine (13)        184        184      0.844      0.881      0.922      0.652
           Canine (23)        

In [3]:
import os
import shutil

folder = '/content/runs'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
os.rmdir(folder)  # now it is empty, so this will work


In [25]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch

import torch

torch.cuda.empty_cache()      # Releases all unused memory
torch.cuda.ipc_collect()      # Releases memory held by inter-process communication


